In [1]:
import plotly.offline as plt
import plotly.graph_objs as go
from plotly.graph_objs import Layout
from plotly.graph_objs.graph_objs import Scatter
from ipywidgets import *
import ipywidgets as widgets

plt.init_notebook_mode(connected=True)

In [2]:
style={'description_width' : '100px'}
class Curves(object):
    """Class for showing scatter plot of intensities for one residue"""
    def __init__(self, titration):
        """Defines attributs
        argument 'titration' is a dicitionnary of 2 dimensions list for each key (should be a titration object from shiftome after developement)"""
        style={'description_width' : '100px'}
        self.titration=titration
        self.residueSelector=widgets.Dropdown(options=[key for key in self.titration],
                                              description='Residue Number',
                                              style = style,
                                              layout = {"width" : '200px'},
                                              disabled=False,) # widget for selecting a specific residue
        self.intensity=widgets.FloatSlider(max=50, 
                                           step=1,
                                           description='Max value',
                                           style = style,
                                           layout = {"width" : '350px'},
                                           orientation="horizontal", 
                                           value=0) # slider widget for setting y ax max value
        self.widgets=VBox([self.residueSelector, self.intensity])
        self.btn = widgets.ToggleButton(value=False,
                                        description='Refresh',
                                        disabled=False,
                                        button_style='',
                                        tooltip='Description',
                                        icon='check')
        self.ratio=[]
        
    def updateAttr(self, ratioValues):
        pass
    def scatterPlot(self, residueNumber, maxValue, refresh):
        """Method that traces scatter plot of intensity at each Ligand/Protein ratio. 
        Creates a scatter plot.Sets the max value. Defines a legend.
        argument 'residueNumber' is a float number representing dictionnary key of titration.
        argument 'maxValue' is an integer representing the max value for y ax"""
        trace=go.Scatter(x=self.ratio, y=self.titration[residueNumber][1],)
        data=[trace]
        ##sets default max value
        if maxValue ==0:
            intensityMax=1.2*max(self.titration[residueNumber][1])
        ##
        ##sets chosen max value
        else:
            intensityMax=maxValue
        ##
        fig = go.Figure(data=data, layout=dict(title='Titration Curve of residue '+str(residueNumber),annotations=[{
                                            'xref': 'paper', #plot number
                                            'yref': 'paper',
                                            'x': 0, #x position of x ax legend
                                            'xanchor': 'right',
                                            'y': 1,
                                            'yanchor': 'bottom',
                                            'text': 'Chemical shift intensity',
                                            'font':{'size':16},
                                            'showarrow': False
                                          }, {
                                            'xref': 'paper',
                                            'yref': 'paper',
                                            'x': 1,
                                            'xanchor': 'left',
                                            'y': 0,
                                            'yanchor': 'top',
                                            'text': 'Ligand/Protein ratio',
                                            'font':{'size':16},
                                            'showarrow': False
                                          }],  width=1000, #width of plotly interface
                                    height=500,
                                    margin=go.Margin(
                                        l=200,
                                        r=200,
                                        b=100,
                                        t=100,
                                        pad=4)),)
        fig['layout'].update(xaxis={'showline':True, 'showgrid':True,'zeroline':False}, 
                             yaxis={'range':[0,intensityMax],'showline':True,'zeroline':False}) #show x and y axes and sets y scale
        plt.iplot(fig)

titration={1:[[1,2,3,4,5,6,7,8],[10,12,14,15,6,20,1,12,]], 
           2:[[1,2,3,4,5,6,7,8],[12,15,14,16,1,21,8,6,]], 
           3:[[1,2,3,4,5,6,7,8],[12,11,10,14,15,16,17,15]], 
           4:[[1,2,3,4,5,6,7,8],[11,14,16,20,25,23,24,48]]}
c=Curves(titration)

In [3]:
style = {'description_width': '200px'}
layout = {'width': '300px'}
#concentration_text = widgets.BoundedFloatText(value=0, description="Etape 0", style=style, layout=layout,disabled=True)
#Concentration_box = widgets.VBox([concentration_text])
#btn = widgets.Button(description = 'Add')
#Add Text Area
#words = ['Volume init', 'Volum fin', 'Concentration titrante', 'Concentration ']
#items = [BoundedFloatText(description=w) for w in words]
#txtArea = widgets.Textarea()
#right_box = VBox([items[2], items[3],items[0], items[1]])
#left_box = VBox([concentration_text,btn, txtArea])



class ProtocoleSteps(object):
    step_nb = 0
    style = {'description_width' : '200px'}
    layout = {"width" : '300px'}
    volume_layout = {"width" : '300px',"margin-top": '120px'}
    btn_layout = {"width" :'150', "margin-top": '20'}
    
    def __init__(self, titration = None):
        
        self.titration = titration
        
        # init volumes widgets
        self.volumesBox = VBox()
        self.volumeWidgets = []
        self.add_volume(disabled = True)
        
        # init concentration widgets
        self.initConcentration = []
        for label in ('[analyte] (µM)', '[titrant] (µM)'):
            widget = widgets.BoundedFloatText(
                value=0, 
                min=0,
                step=0.01,
                description=label, 
                style=self.style, 
                layout=self.layout,
                disabled=False
            )
            self.initConcentration.append(widget)
            
        # init start volumes widgets
        self.initVolumes = []
        for label in ('Analyte start volume', 'Total volume'):
            widget = widgets.BoundedFloatText(
                value=0, 
                min=0,
                step=0.01,
                description=label,
                style=self.style, 
                layout=self.volume_layout,
                disabled=False,
            )
            self.initVolumes.append(widget)
        
        # layout initial parameters
        volumeBox = VBox(self.initVolumes)
        concentrationBox = VBox(self.initConcentration)
        self.initBox = HBox([volumeBox, concentrationBox])
        
        
        
        self.btn = widgets.Button(description = 'Add', style=style, layout=self.btn_layout)
        self.btn.on_click(self.on_btn_clicked)
        
        
        
        #self.right_box = VBox([self.items[0], self.items[1],self.items[2], self.items[3]])
        #self.left_box = VBox([self.concentration_text,self.btn])
        
        self.mainBox = VBox([self.initBox, self.volumesBox, self.btn])
        self.ratio=[]
        
    @property
    def volumes(self):
        return [volWidget.value for volWidget in self.volumeWidgets]
        
    def on_btn_clicked(self,b):
        self.add_volume()

    def volume_changed(self, change):
        startVolumes=self.initBox.children[0].children
        concentrations=self.initBox.children[1].children
        addedVolumes=self.volumeWidgets
        try:
            analyteConc=[(startVolumes[0].value/startVolumes[1].value)*concentrations[0].value for step in addedVolumes]
            titrantVol=[volume.value for volume in addedVolumes]
            titrantConc=[(sum(titrantVol[0:index+1])/startVolumes[1].value)*concentrations[1].value for index,step in enumerate(addedVolumes)]
        except ZeroDivisionError as e:
            print(str(e) + ": Total volume = 0 !")
            return
        try:
            ratioConc=[titrantConc[index]/analyteConc[index] for index in range(len(addedVolumes)) ]
            print("Vol Changed ! Ratio L/P: "+str(ratioConc))
            self.ratio=ratioConc 
        except ZeroDivisionError as e:
            print(str(e) + ": Analyte concentration = 0 !")
            return

        
    def add_volume(self, disabled=False):
        widget = widgets.BoundedFloatText(
            value=0, 
            min=0,
            step=0.01,
            description="Step {step}".format(step=self.step_nb), 
            style=self.style, 
            layout=self.layout,
            disabled=disabled
        )
        
        widget.observe(self.volume_changed, names='value')
        
        self.volumeWidgets.append(widget)
        self.volumesBox.children = self.volumeWidgets
        self.step_nb += 1

protocole = ProtocoleSteps()
display(protocole.mainBox)

def updateCurveAx(b):
    if protocole.ratio!=[]:
        b.value=protocole.ratio
        c.ratio=protocole.ratio
        print("Ratio values have been updated !\nRefresh your graph.")
    else:
        print("Emtpy ratio values.")

linkBtn = widgets.Button(description = 'Validate ratio values', style=style, layout=layout)
linkBtn.on_click(updateCurveAx)

display(linkBtn)

A Jupyter Widget

A Jupyter Widget

Vol Changed ! Ratio L/P: [0.0, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]
Vol Changed ! Ratio L/P: [0.0, 0.2, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6]
Vol Changed ! Ratio L/P: [0.0, 0.2, 0.6, 1.4, 1.4, 1.4, 1.4, 1.4, 1.4]
Vol Changed ! Ratio L/P: [0.0, 0.2, 0.6, 1.4, 2.9999999999999996, 2.9999999999999996, 2.9999999999999996, 2.9999999999999996, 2.9999999999999996]
Vol Changed ! Ratio L/P: [0.0, 0.02, 0.06, 0.14, 0.3, 0.5, 0.5, 0.5, 0.5]
Vol Changed ! Ratio L/P: [0.0, 0.02, 0.06, 0.14, 0.3, 0.5, 0.7500000000000001, 0.7500000000000001, 0.7500000000000001]
Vol Changed ! Ratio L/P: [0.0, 0.02, 0.06, 0.14, 0.3, 0.5, 0.7500000000000001, 1.05, 1.05]
Vol Changed ! Ratio L/P: [0.0, 0.02, 0.06, 0.14, 0.3, 0.5, 0.7500000000000001, 1.05, 1.43]
Vol Changed ! Ratio L/P: [0.0, 0.008, 0.024, 0.056, 0.12, 0.2, 0.30000000000000004, 0.4200400000000001, 0.57204]
Vol Changed ! Ratio L/P: [0.0, 0.008, 0.024, 0.056, 0.12, 0.2, 0.30000000000000004, 0.42000000000000004, 0.572]
Vol Changed ! Ratio L/P: [0.0, 0.008, 0.0

In [4]:

widgets.interactive(c.scatterPlot, residueNumber=c.widgets.children[0], maxValue=c.widgets.children[1], refresh=c.btn)

A Jupyter Widget